In [ ]:
"""*****************************************************************************************
MIT License
Copyright (c) 2021 Jyotikrishna Dass and Rabi Mahapatra
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
*****************************************************************************************"""


"""####################################### NOTES ############################################
# - Please cite our paper when using the code: 
#       "Householder Sketch for Accurate and Accelerated Least-Mean-Squares Solvers" (ICML 2021)
#                          Jyotikrishna Dass and Rabi Mahapatra
##########################################################################################"""

In [2]:
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import KFold
import time
import math
import warnings
import scipy.linalg.lapack as LAPACK
import scipy
import pandas as pd

In [3]:
###################################################################################
#             "Fast and Accurate Least-Mean-Squares Solvers" 
#        (NeurIPS19' - Oral presentation, Outstanding Paper Honorable Mention) 
#             Alaa Maalouf and Ibrahim Jubran and Dan Feldman
#                    Open source code released by Authors
#  https://github.com/ibramjub/Fast-and-Accurate-Least-Mean-Squares-Solvers/blob/master/Booster.py
#   PAPER:   https://papers.nips.cc/paper/2019/file/475fbefa9ebfba9233364533aafd02a3-Paper.pdf

#
# We thank the authors for their open-source code and inspiring us through their work
###################################################################################

def Caratheodory(P, u, dtype='float64'):
    """
    Implementation of the Caratheodory Theorem(1907)
    input: a numpy array P containing n rows (points), each of size d, and a positive vector of weights u (that sums to 1)
    output:a new vector of weights new_u that satisfies :
                1. new_u is positive and sums to 1
                2. new_u has at most d+1 non zero entries
                3. the weighted sum of P and u (input) is the same as the weighted sum of P and new_u (output)
    computation time: O(n^2d^2)
    """
    while 1:
        n = np.count_nonzero(u)
        d = P.shape[1]
        u_non_zero = np.nonzero(u)

        if n <= d + 1:
            return u

        A = P[u_non_zero]
        reduced_vec = np.outer(A[0], np.ones(A.shape[0]-1, dtype = dtype))
        A = A[1:].T - reduced_vec

        _, _, V = np.linalg.svd(A, full_matrices=True)
        v=V[-1]
        v = np.insert(v, [0],   -1 * np.sum(v))

        idx_good_alpha = np.nonzero(v > 0)
        alpha = np.min(u[u_non_zero][idx_good_alpha]/v[idx_good_alpha])

        w = np.zeros(u.shape[0] , dtype = dtype)
        tmp_w = u[u_non_zero] - alpha * v
        tmp_w[np.argmin(tmp_w)] = 0.0
        w[u_non_zero] = tmp_w
        w[u_non_zero][np.argmin(w[u_non_zero] )] = 0
        u = w

def Fast_Caratheodory(P,u,coreset_size, dtype = 'float64'):
    """
    Our fast and accurate implementation of Caratheodory's Theorem
    Input: a numpy array P containing n rows (points), each of size d, and a positive vector of weights u (if u does not
    sum to 1, we first normalize u by its sum, then multiply u back by its original sum before returning it)
    Output: a new vector of positive weights new_u that satisfies :
                 1. new_u has at most d+1 non zero entries
                 2. the weighted sum of P and u (input) is the same as the weighted sum of P and new_u (output)
    Computation time: O(nd+logn*d^4)
    """
    d = P.shape[1]
    n = P.shape[0]
    m = 2*d + 2
    if n <= d + 1:
        return u.reshape(-1)

    u_sum = np.sum(u)
    u = u/u_sum
    chunk_size = math.ceil(n/m)
    current_m = math.ceil(n/chunk_size)

    add_z = chunk_size - int (n%chunk_size)
    u = u.reshape(-1,1)
    if add_z != chunk_size:
        zeros = np.zeros((add_z, P.shape[1]), dtype = dtype)
        P = np.concatenate((P, zeros))
        zeros = np.zeros((add_z, u.shape[1]), dtype = dtype)
        u = np.concatenate((u, zeros))
    
    idxarray = np.array(range(P.shape[0]) )
    
    p_groups = P.reshape(current_m, chunk_size, P.shape[1])
    u_groups = u.reshape(current_m, chunk_size)
    idx_group = idxarray.reshape(current_m, chunk_size)
    u_nonzero = np.count_nonzero(u)

    if not coreset_size:
        coreset_size = d+1
    while u_nonzero > coreset_size:

        groups_means = np.einsum('ijk,ij->ik',p_groups, u_groups)
        group_weigts = np.ones(groups_means.shape[0], dtype = dtype)*1/current_m

        Cara_u_idx = Caratheodory(groups_means , group_weigts,dtype = dtype )

        IDX = np.nonzero(Cara_u_idx)

        new_P = p_groups[IDX].reshape(-1,d)

        subset_u = (current_m * u_groups[IDX] * Cara_u_idx[IDX][:, np.newaxis]).reshape(-1, 1)
        new_idx_array = idx_group[IDX].reshape(-1,1)
        ##############################################################################
        u_nonzero = np.count_nonzero(subset_u)
        chunk_size = math.ceil(new_P.shape[0]/ m)
        current_m = math.ceil(new_P.shape[0]/ chunk_size)

        add_z = chunk_size - int(new_P.shape[0] % chunk_size)
        if add_z != chunk_size:
            new_P = np.concatenate((new_P, np.zeros((add_z, new_P.shape[1]), dtype = dtype)))
            subset_u = np.concatenate((subset_u, np.zeros((add_z, subset_u.shape[1]),dtype = dtype)))
            new_idx_array = np.concatenate((new_idx_array, np.zeros((add_z, new_idx_array.shape[1]),dtype = dtype)))
        p_groups = new_P.reshape(current_m, chunk_size, new_P.shape[1])
        u_groups = subset_u.reshape(current_m, chunk_size)
        idx_group = new_idx_array.reshape(current_m , chunk_size)
        ###########################################################

    new_u = np.zeros(n)
    subset_u = subset_u[(new_idx_array < n)]
    new_idx_array = new_idx_array[(new_idx_array < n)].reshape(-1).astype(int)
    new_u[new_idx_array] = subset_u
    return u_sum * new_u


def linregcoreset(P, u, b=None, c_size=None, dtype='float64'):
    """
    This function computes a coreset for linear regression.
    Input: a numpy array P containing n rows (points), each of size d, a positive vector of weights u of size n, a labels
           vector b of size n, coreset size c_size (not required).
    Output: a new numpy array new_P containing the coreset points in its rows and a new vector new_u of positive weights,
            and a new vector of labels new_b for the coreset. The output satisfies for every vector x that:
                 ||sqrt(u.transpose())*(Px-b)||^2 = ||sqrt(new_u.transpose())*(new_Px-new_b)||^2
                 i.e., the output of a call to linearRegression with the original input or with the coreset is the same.
    Computation time: O(nd^2+logn*d^8)
    """
    if b is not None:
        P_tag = np.append(P, b, axis=1)
    else:
        P_tag = P

    n_tag = P_tag.shape[0]; d_tag = P_tag.shape[1]
    P_tag = P_tag.reshape(n_tag, d_tag, 1)
    
    P_tag = np.einsum("ikj,ijk->ijk",P_tag ,P_tag)
    P_tag = P_tag.reshape(n_tag, -1)
    n_tag = P_tag.shape[0]; d_tag = P_tag.shape[1]

    coreset_weigts = Fast_Caratheodory(P_tag.reshape(n_tag,-1), u, c_size,  dtype=dtype)
    new_idx_array = np.nonzero(coreset_weigts)
    coreset_weigts = coreset_weigts[new_idx_array]

    if b is not None:
        return P[new_idx_array], coreset_weigts.reshape(-1), b[new_idx_array]
    else:
        return P[new_idx_array], coreset_weigts.reshape(-1)


def stream_coreset(P, u, b, folds=None, dtype='float64'):
    """
    This function computes a coreset for LMS solvers that use k-fold cross validation. It partitions the data into "folds"
    parts, and computes a coreset for every part using the function linregcoreset.
    Input: a numpy array P containing n rows (points), each of size d, a positive vector of weights u of size n, a labels
           vector b of size n, and the number of folds used in the cross validation.
    Output: a new numpy array new_P containing the coreset points in its rows and a new vector new_u of positive weights,
            and a new vector of labels new_b for the coreset. The output satisfies for every vector x that:
                 ||sqrt(u.transpose())*(Px-b)||^2 = ||sqrt(new_u.transpose())*(new_Px-new_b)||^2
                 i.e., the output of a call to linearRegression with the original input or with the coreset is the same.
    Computation time: O(nd^2+logn*d^8)
    """
    if folds is None:
        return linregcoreset(P, u, b, dtype=dtype)
    m = int(P.shape[0] / folds)

    d = P.shape[1]
    size_of_coreset = ((d+1)*(d+1)+1)

    batches = folds
    cc, uc, bc = linregcoreset(P[0:m], u[0:m], b[0:m], dtype=dtype)

    if cc.shape[0] < size_of_coreset and folds:
            add_z = size_of_coreset - cc.shape[0]
            zeros = np.zeros((add_z, cc.shape[1]), dtype=dtype)
            cc = np.concatenate((cc, zeros))
            zeros = np.zeros((add_z), dtype=dtype)
            uc = np.concatenate((uc, zeros))
            zeros = np.zeros((add_z, bc.shape[1]), dtype=dtype)
            bc = np.concatenate((bc, zeros))

    for batch in range(1, batches):
        coreset, new_u, new_b = linregcoreset(P[batch*m:(batch+1)*m], u[batch*m:(batch+1)*m], b[batch*m:(batch+1)*m], dtype=dtype)

        if coreset.shape[0] < size_of_coreset and folds:
            add_z = size_of_coreset - coreset.shape[0]
            zeros = np.zeros((add_z, coreset.shape[1]), dtype=dtype)
            coreset = np.concatenate((coreset, zeros))
            zeros = np.zeros((add_z),dtype=dtype)
            new_u = np.concatenate((new_u, zeros))
            zeros = np.zeros((add_z, new_b.shape[1]), dtype=dtype)
            new_b = np.concatenate((new_b, zeros))
        bc = np.concatenate((bc, new_b))
        cc = np.concatenate((cc, coreset))
        uc = np.concatenate((uc, new_u))
    return cc, uc, bc


def test_model_(test_data, test_labels, test_weights, clf):
    weighted_test_data = test_data * np.sqrt(test_weights[:, np.newaxis])
    weighted_test_labels = test_labels * np.sqrt(test_weights[:, np.newaxis])
    score = clf.score(weighted_test_data, weighted_test_labels)
    return score


def train_model_(data, labels, weights, clf):
    time_start = time.time()
    weighted_data = data * np.sqrt(weights[:, np.newaxis])
    weighted_labels = (labels * np.sqrt(weights[:, np.newaxis])).ravel()
    clf.fit(weighted_data, weighted_labels)
    time_end = time.time()

    return time_end - time_start, clf


def coreset_train_model(data, labels, weights, clf, folds=None, solver='ridge'):
    time_start = time.time()

    coreset, coreset_weights, coreset_labels = stream_coreset(data, weights, labels, folds=folds)
    weighted_coreset = coreset * np.sqrt(coreset_weights[:, np.newaxis])
    weighted_coreset_labels = (coreset_labels * np.sqrt(coreset_weights[:, np.newaxis])).ravel()

    if solver in ['lasso', 'elastic']:
        const = np.sqrt(coreset.shape[0] / data.shape[0])
        clf.fit(const * weighted_coreset, const * weighted_coreset_labels)
    else:
        clf.fit(weighted_coreset, weighted_coreset_labels)
    time_end = time.time()

    return time_end - time_start, clf

###################################################################################

In [4]:
###################################################################################
#       "Householder Sketch for Accurate and Accelerated Least-Mean-Squares Solvers"
#                   Jyotikrishna Dass and Rabi Mahapatra (ICML 2021)      
#                               (Sequential LMS-QR)
###################################################################################

def train_model(data, labels, clf):
  time_start = time.time()
  clf.fit(data,labels.ravel())
  time_end = time.time()
  return time_end - time_start, clf

def test_model(test_data, test_labels,clf):
  score = clf.score(test_data, test_labels.ravel())
  return score

def qrHouseholder(X):  # LAPACK.dgeqrf
  time_start = time.time()
  output = LAPACK.dgeqrf(X)
  H = output[0]
  tau = output[1]
  R = np.triu(H[:X.shape[1], :])
  time_end = time.time()
  return time_end - time_start,H, tau, R

def multiplyQx(H, tau, x, side='L', trans='N'):
  # http://www.netlib.org/lapack/explore-html/da/dba/group__double_o_t_h_e_rcomputational_ga17b0765a8a0e6547bcf933979b38f0b0.html
  # https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lapack.dormqr.html#scipy.linalg.lapack.dormqr
  # side: 'L' (Q*x)  | 'R' (x*Q)    (Puts Q on left or right side of x)
  # trans: 'N' (Q*x) | 'T' (x'*Q)   (transposes Q)
  # tau is already defined
  # x is also aready defined.
  time_start = time.time()
  output = LAPACK.dormqr(side=side, trans=trans, a=H, tau=tau, c=x, lwork=x.shape[0])
  out_clean = np.asarray_chkfinite(output[:-2])[0]
  time_end = time.time()
  return time_end - time_start,out_clean
###################################################################################

In [5]:
###################################
# LMS solvers from scikit-learn
###################################

def get_new_clf(solver, folds=3, alphas=100):
  kf=KFold(n_splits=folds,shuffle=False)   
  if "linear" == solver:
    clf = linear_model.LinearRegression(fit_intercept=False)

  if "ridge" == solver:
    alphas =  np.arange(1/alphas, 10+ 1/alphas, 10/alphas)
    clf = linear_model.RidgeCV(alphas=alphas, fit_intercept=False, cv=kf)

  elif "lasso" == solver:
    clf=linear_model.LassoCV(n_alphas=alphas, fit_intercept=False, cv=kf)

  elif "elastic" == solver:
    clf = linear_model.ElasticNetCV(n_alphas=alphas, fit_intercept=False, cv=kf)

  elif "kernelRidge" == solver:
    clf = KernelRidge(alpha=0.1, kernel="linear")

  return clf

In [6]:
###################################
# main() 
###################################

def main():

  # SYNTHETIC DATASETS
  n = 24000000
  d = 3
  data_range = 100
  np.random.seed(0)
  data = np.floor(np.random.rand(n, d) * data_range) #dtype='float32'
  labels = np.floor(np.random.rand(n, 1) * data_range)
  num_of_alphas = 100
  folds = 2

  # REAL DATASETS - (1)
  # dataset = pd.read_pickle("3D_Spatial_network.zip")
  # dataset.dropna(inplace=True)
  # data = dataset[['LONGITUDE', 'LATITUDE']].values
  # labels = dataset[['ALTITUDE']].values
  # n, d = data.shape
  # num_of_alphas = 100
  # folds = 2

  # REAL DATASETS - (2)
  # dataset = pd.read_pickle("household_power.zip")
  # dataset.dropna(inplace=True)
  # #ns = 250000
  # #data = dataset[['Global_active_power', 'Global_reactive_power']].iloc[:ns, :].values
  # #labels = dataset[['Voltage']].iloc[:ns, :].values
  # data = dataset[['Global_active_power', 'Global_reactive_power']].values
  # labels = dataset[['Voltage']].values
  # n, d = data.shape
  # num_of_alphas = 100
  # folds = 2


  weights = np.ones(n)    
  
  #solverList = ["ridge", "lasso", "elastic", "linear"]
  
  solverList = ["ridge"]
  for solver in solverList:

    print("SOLVER: {} \nData: {} X {} \nnumber_of_alphas: {}".format(solver,n,d,num_of_alphas))



    print("\n################ Original (no optimization) #################")
    
    clf = get_new_clf(solver, folds=folds, alphas=num_of_alphas)
    time_og, clf_og = train_model(data, labels, clf) 
    print("coef_original:\n {}\n".format(clf_og.coef_))
    print("*** RUNNING TIME ***")
    print ("---->time_original = {} \n".format(time_og))
    


    print("\n#################### LMSQR (Householder) ###################")
    
    warnings.filterwarnings("ignore")
    time_preprocess = 0
    time_train = 0
    time_hh, H, tau, R = qrHouseholder(data)  # LAPACK Householder QR for input data
    time_Qx, qtrans_labels = multiplyQx(H, tau, labels, trans="T")
    time_preprocess = time_hh + time_Qx 
    clf = get_new_clf(solver, folds=folds, alphas=num_of_alphas)
    time_train, clf_qr = train_model(R, qtrans_labels[0:d],clf)
    time_LMSQR = time_preprocess + time_train
    print("coef_qr:\n {}, coef diff wrt original = {}\n".format(clf_qr.coef_,np.sum(np.abs(clf_og.coef_ - clf_qr.coef_)))) 
    print("*** RUNNING TIME ***")
    print("---->time_LMSQR (Preprocess+Train) = {}".format(time_LMSQR))
    print("    ---->time(Preprocess=A+B) = {}".format(time_preprocess))
    print("         -A-->time(Householder) = {}".format(time_hh))
    print("         -B-->time(Qx) = {}".format(time_Qx))
    print("    ---->time(Train) = {}\n".format(time_train))
    print("*** RUNNING TIME SPEEDUP ***")
    print("LMSQR wrt original = {} X".format(time_og/time_LMSQR))



    print("\n################## LMSBOOST (Caratheodory) ####################")
    
    clf = get_new_clf(solver, folds=folds, alphas=num_of_alphas)
    time_LMSBOOST, clf_caratheodory = coreset_train_model(data, labels, weights, clf, folds=folds, solver=solver)
    print("coef_caratheodory:\n {}, coef diff wrt original = {}\n".format(clf_caratheodory.coef_, np.sum(np.abs(clf_og.coef_ - clf_caratheodory.coef_))))
    print("*** RUNNING TIME ***")
    print ("---->time_LMSBOOST = {}\n".format(time_LMSBOOST))
    print("*** RUNNING TIME SPEEDUP ***")
    print("LMSBOOST wrt original = {} X".format(time_og/time_LMSBOOST))
    print("LMSQR wrt LMSBOOST = {} X".format(time_LMSBOOST/time_LMSQR))
    
    
    if (solver == "linear"):
      print("\n####################### SKETCH+INVERSE #########################")
      time_start = time.time()
      invX = np.linalg.inv(np.transpose(data)@data)
      clf_sketchinv_coef_ = invX @ (np.transpose(data)@labels)
      time_end = time.time()
      time_SKETCHINV = time_end - time_start
      print("coef_SKETCH+INVERSE:\n {}, coef diff wrt original = {}\n".format(clf_sketchinv_coef_, np.sum(np.abs(clf_og.coef_ - clf_sketchinv_coef_))))
      print("*** RUNNING TIME ***")
      print ("---->time_SKETCH+INVERSE = {}\n".format(time_SKETCHINV))
      print("*** RUNNING TIME SPEEDUP ***")
      print("SKETCH+INVERSE wrt original = {} X".format(time_og/time_SKETCHINV))
      print("LMSQR wrt SKETCH+INVERSE = {} X".format(time_SKETCHINV/time_LMSQR))


      print("\n####################### SKETCH+CHOLESKY ##########################")
      time_start = time.time()
      L = np.linalg.cholesky(np.transpose(data)@data)
      invLLt = np.linalg.inv(L@np.transpose(L))
      clf_sketchchol_coef_ = invLLt @ (np.transpose(data)@labels)
      time_end = time.time()
      time_SKETCHCHOLESKY = time_end - time_start
      print("coef_SKETCH+CHOLESKY:\n {}, coef diff wrt original = {}\n".format(clf_sketchchol_coef_, np.sum(np.abs(clf_og.coef_ - clf_sketchchol_coef_))))
      print("*** RUNNING TIME ***")
      print ("---->time_SKETCH+CHOLESKY= {}\n".format(time_SKETCHCHOLESKY))
      print("*** RUNNING TIME SPEEDUP ***")
      print("SKETCH+CHOLESKY wrt original = {} X".format(time_og/time_SKETCHCHOLESKY))
      print("LMSQR wrt SKETCH+CHOLESKY = {} X".format(time_SKETCHCHOLESKY/time_LMSQR))
 
   
   
      print("-------------------------------------------------------------------------------------\n")

   
      ################ ACCURACY COMPARISON #############################
      #######  defined in Maalouf et al (NeurIPS2019) ##################
      ##########   for LMS = LinearRegression    #############################
      ##################################################################

      print("\n*** ACCURACY COMPARISON ***\n")

      # Original, w* = LinearRegression(X, b) and ||Xw*-b||
      loss_og = np.linalg.norm(data@clf_og.coef_ - labels.ravel())
      print("Loss original = {} ".format(loss_og))
        

      # Householder QR: w_qr =  LinearRegression(R, Q'b) and loss = ||Xw_qr-b||
      loss_qr = np.linalg.norm(data@clf_qr.coef_ - labels.ravel())
      print("Loss LMSQR = {} ". format(loss_qr))
      #print("Difference in loss LMSQR = {} ". format(np.abs(loss_og-loss_qr)))


      # Caratheodory set: w_caratheodory = LinearRegression(C, y) and loss = ||Xw_caratheodory-b||
      loss_caratheodory = np.linalg.norm(data@clf_caratheodory.coef_ - labels.ravel())
      print("Loss LMSBOOST = {} ". format(loss_caratheodory))
      #print("Difference in loss LMSBOOST = {} ". format(np.abs(loss_og-loss_caratheodory)))


      # SKETCH+INVERSE (for linear solver), w_sketchinv =  (X'X)^{-1}X'b , and ||Xw_sketchinv-b||
      loss_sketchinv = np.linalg.norm(data@clf_sketchinv_coef_ - labels)
      print("Loss SKETCH+INVERSE = {} ". format(loss_sketchinv))
      #print("Difference in loss SKETCH+INVERSE = {} ". format(np.abs(loss_og-loss_sketchinv)))


      # SKETCH+CHOLESKY (for linear solver), w_sketchchol = (LL')^{-1}A'b  and ||Xw_sketchchol-b||
      loss_sketchchol = np.linalg.norm(data@clf_sketchchol_coef_ - labels)
      print("Loss SKETCH+CHOLESKY = {} ". format(loss_sketchchol))
      #print("Difference in loss SKETCH+CHOLESKY = {} ". format(np.abs(loss_og-loss_sketchchol)))
  
    
      print("-------------------------------------------------------------------------------------\n")

      # ############## Numerical Stability ##################
      #######  defined in Maalouf et al (NeurIPS2019) ##################
      ##########   for LMS = LinearRegression    #############################
      ##################################################################
    
      print("\n*** NUMERICAL STABILITY (lower is better) ***\n")

      # Householder QR: w_qr =  LinearRegression(R, Q'b) and ||Xw* - Xw_qr||
      stability_qr = np.linalg.norm((data@clf_og.coef_) - (data@clf_qr.coef_))
      print("LMSQR wrt Original: {:.20}".format(stability_qr))


      # Caratheodory set: w_caratheodory = LinearRegression(C, y) and ||Xw* - Xw_caratheodory||
      stability_caratheodory = np.linalg.norm((data@clf_og.coef_) - (data@clf_caratheodory.coef_))
      print("LMSBOOST wrt Original: {:.20}".format(stability_caratheodory))


      # SKETCH+INVERSE: w_sketchinv =  (X'X)^{-1}X'b , and ||Xw* - Xw_sketchinv||
      stability_sketchinv = np.linalg.norm((data@clf_og.coef_) - (data@clf_sketchinv_coef_))
      print("SKETCH+INVERSE wrt Original: {:.20}".format(stability_sketchinv))


      # SKETCH+CHOLESKY, w_sketchchol = (LL')^{-1}A'b  and ||Xw* - Xw_sketchchol||
      stability_sketchchol = np.linalg.norm((data@clf_og.coef_) - (data@clf_sketchchol_coef_))
      print("SKETCH+CHOLESKY wrt Original: {:.20}".format(stability_sketchchol))
    


if __name__ == '__main__':
  main()
  print("------------------------------------  END   -------------------------------------\n")

SOLVER: ridge 
Data: 24000000 X 3 
number_of_alphas: 100

################ Original (no optimization) #################
coef_original:
 [0.29931347 0.29917227 0.29975626]

*** RUNNING TIME ***
---->time_original = 196.09504675865173 


#################### LMSQR (Householder) ###################
coef_qr:
 [0.29931347 0.29917227 0.29975626], coef diff wrt original = 2.7755575615628914e-15

*** RUNNING TIME ***
---->time_LMSQR (Preprocess+Train) = 1.1159121990203857
    ---->time(Preprocess=A+B) = 0.8937475681304932
         -A-->time(Householder) = 0.5483131408691406
         -B-->time(Qx) = 0.34543442726135254
    ---->time(Train) = 0.22216463088989258

*** RUNNING TIME SPEEDUP ***
LMSQR wrt original = 175.72623270074084 X

################## LMSBOOST (Caratheodory) ####################
coef_caratheodory:
 [0.29931347 0.29917227 0.29975626], coef diff wrt original = 1.3629652961810734e-12

*** RUNNING TIME ***
---->time_LMSBOOST = 10.388382911682129

*** RUNNING TIME SPEEDUP ***
LMSBOO